In [1]:
# WARNING! Be sure to change the line that reads the .csv file, below, should be line 47
# If it doesn't work, you'll need to make the path point to your copy of that file (which should be in this folder)

# We need to import these modules to get started
import numpy as np
import pandas as pd
import os
import sys
import matplotlib.pyplot as plt
from allensdk.core.brain_observatory_cache import BrainObservatoryCache

# The dynamic brain database is on an external hard drive.
#drive_path = '/Volumes/Brain2016'
drive_path= 'd:'
manifest_path = os.path.join(drive_path, 'BrainObservatory/manifest.json')
boc = BrainObservatoryCache(manifest_file=manifest_path)
expt_containers = boc.get_experiment_containers()

# Things I will want eventually:
# 1. A dataframe with columns for...
#	a. all the identifiers from the brain observatory
#	b. the preferred directions from the grating and place field stimuli
# 2. A numpy array which is N_neurons x N_timesamples, where entry i,j is 1 or 0 based on whether
#	 the neuron was "firing" at that point (how we decide that we'll get to later

# Data frames first:
# expt_containers_df = pd.DataFrame(expt_containers)

# The stimuli I'm interested in are basically everything except the natural movie and natural scenes
# If we want other stimuli, add them here.
non_movie_stimuli = ['drifting_gratings', 'locally_sparse_noise', 'spontaneous', 'static_gratings']

all_expts_df = pd.DataFrame(boc.get_ophys_experiments(stimuli=non_movie_stimuli))
# this has headers:
# age_days	cre_line	experiment_container_id	id	imaging_depth	session_type	targeted_structure

# seems like I can use get_cell_speciments to get everything I'm after
specimens_df = pd.DataFrame(boc.get_cell_specimens(experiment_container_ids=all_expts_df.experiment_container_id.values))
# this has headers:
# area	cell_specimen_id	dsi_dg	experiment_container_id	imaging_depth	osi_dg	osi_sg	p_dg	p_ns	p_sg
# pref_dir_dg	pref_image_ns	pref_ori_sg	pref_phase_sg	pref_sf_sg	pref_tf_dg	time_to_peak_ns	time_to_peak_sg
# tld1_id	tld1_name	tld2_id	tld2_name	tlr1_id	tlr1_name

# There's also a handy bit of data from Saskia, in the form of a measurement called S. See
# Decoding Visual Inputs From Multiple Neurons in the Human Temporal Lobe, J. Neurophys 2007, by Quiroga et al


selectivity_S_df = pd.read_csv(os.path.join(drive_path, '/BrainObservatory/image_selectivity_dataframe.csv'), index_col=0)
selectivity_S_df = selectivity_S_df[['cell_specimen_id', 'selectivity_ns']]

specimens_with_selectivity_S = specimens_df.merge(selectivity_S_df,how='outer', on='cell_specimen_id')

# This is all cells in VISp that have a value for the specified parameters (i.e not NaN)
# Discards rows NaN in the columns specified below.
discard_nan = [
	'selectivity_ns',
	'osi_sg',
	'osi_dg',
	'time_to_peak_ns',
	'time_to_peak_sg',
]

In [2]:
VIS_cells_with_numbers = specimens_with_selectivity_S[specimens_with_selectivity_S.area == 'VISp']
for col_name in discard_nan:
	VIS_cells_with_numbers = VIS_cells_with_numbers[np.isnan(VIS_cells_with_numbers[col_name])==False]
    

In [3]:
exp_conts = VIS_cells_with_numbers["experiment_container_id"].unique()
print (exp_conts)

all_tables = {ID:VIS_cells_with_numbers[VIS_cells_with_numbers["experiment_container_id"]==ID]
for ID in exp_conts}

all_tables[511498742].head()

[511498742 511506664 511507144 511507811 511507650 511509529 511510650
 511510718 511510670 511510884 511510955 511510911 512124562 511510758
 511510736 511510667 511510896 511510927 511510974 511510855 511510699
 511510779 511510675 511510989 511510635]


,area,cell_specimen_id,dsi_dg,experiment_container_id,imaging_depth,osi_dg,osi_sg,p_dg,p_ns,p_sg,...,pref_tf_dg,time_to_peak_ns,time_to_peak_sg,tld1_id,tld1_name,tld2_id,tld2_name,tlr1_id,tlr1_name,selectivity_ns
381,VISp,517397361,0.697331,511498742,350,1.388875,1.210908,0.000002,3.065294e-18,1.111408e-01,...,1.0,0.33250,0.26600,177837516,Scnn1a-Tg3-Cre,177837320,Camk2a-tTA,265943423,Ai93(TITL-GCaMP6f),0.586288
382,VISp,517397365,3.640992,511498742,350,0.907149,1.994455,0.002307,2.121158e-13,7.670954e-03,...,1.0,0.29925,0.26600,177837516,Scnn1a-Tg3-Cre,177837320,Camk2a-tTA,265943423,Ai93(TITL-GCaMP6f),0.513627
383,VISp,517397369,1.818350,511498742,350,0.793207,0.829441,0.000088,1.675675e-11,8.678658e-05,...,1.0,0.16625,0.16625,177837516,Scnn1a-Tg3-Cre,177837320,Camk2a-tTA,265943423,Ai93(TITL-GCaMP6f),0.150966
384,VISp,517397373,0.410803,511498742,350,2.074779,0.588888,0.002205,1.632841e-23,1.870294e-07,...,15.0,0.29925,0.19950,177837516,Scnn1a-Tg3-Cre,177837320,Camk2a-tTA,265943423,Ai93(TITL-GCaMP6f),0.462508
385,VISp,517397377,1.176431,511498742,350,1.317674,1.070871,0.007289,1.429351e-07,1.556623e-12,...,2.0,0.29925,0.29925,177837516,Scnn1a-Tg3-Cre,177837320,Camk2a-tTA,265943423,Ai93(TITL-GCaMP6f),0.385254


In [4]:
#print(VIS_cells_with_numbers.keys())

In [5]:
#print(VIS_cells_with_numbers["pref_image_ns"].unique())
#len(VIS_cells_with_numbers["pref_image_ns"].unique())

In [6]:
# very long, uncomment only if you wish to see number of cells that prefer EACH image

#print("# of total cells according to their preferred image: ")
#for image in sorted(VIS_cells_with_numbers["pref_image_ns"].unique()):
#    print ("prefer image #"+str(int(image))+":  "+str(len(VIS_cells_with_numbers
#                                         [VIS_cells_with_numbers["pref_image_ns"]==image]))+" cells")

In [7]:
image_list =[]

for exID in exp_conts:
    df = all_tables[exID]

    #print("cell totals per preferred image group:") 
   
    image_ns_prefs = [len(df[df["pref_image_ns"]==image])
                      for image in sorted(df["pref_image_ns"].unique())]
    
    #print(image_ns_prefs) #list of cells
    #print('')
    #print("total cells in all groups:")
    total_ns_pref = sum(image_ns_prefs)
    #print(total_ns_pref)
    #print('')

    #print("largest group of cells:")
    max_ns_prefs = np.max(image_ns_prefs) #max number cells with preferred static image       
    #print(max_ns_prefs)
    #print('')

    #print("percent of total cells:")
    per_ns_pref = (float(max_ns_prefs)/float(total_ns_pref))*100
    #print (str(per_ns_pref)+ " = roughly "+str(round(per_ns_pref*100))+"%")
    #print("")

    #print("most preferred image(s):")
    for image in sorted(df["pref_image_ns"].unique()):
        if(((len(df[df["pref_image_ns"]==image])))==max_ns_prefs):                
            pref_image=int(image)
            
    image_dict = {"exID":exID,"total cells":total_ns_pref,"max cells":max_ns_prefs,"percent":per_ns_pref,"preferred image":pref_image}
    
    image_list.append(image_dict)
    
image_list[0]

{'exID': 511498742,
 'max cells': 10,
 'percent': 6.289308176100629,
 'preferred image': 29,
 'total cells': 159}

In [10]:
#Percentages of total cells that prefer popular image

percent_list = [percent["percent"] for percent in image_list]
print(percent_list)

[6.289308176100629, 8.974358974358974, 5.555555555555555, 3.7593984962406015, 4.901960784313726, 4.761904761904762, 5.5045871559633035, 6.557377049180328, 4.954954954954955, 8.411214953271028, 8.536585365853659, 6.2857142857142865, 5.88235294117647, 6.122448979591836, 5.454545454545454, 4.571428571428571, 6.382978723404255, 4.3478260869565215, 4.195804195804196, 4.597701149425287, 4.602510460251046, 4.435483870967742, 4.66321243523316, 13.043478260869565, 7.4074074074074066]


In [12]:
#Total pop images - just a check
pop_image_list = [pop_image["preferred image"] for pop_image in image_list]
print(len(pop_image_list))

# Unique popular images
unique_images = set(pop_image_list)
print(len(unique_images))

#Popular image IDs
print (pop_image_list)

25
16
[29, 110, 29, 105, 49, 24, 86, 43, 85, 56, 85, 25, 111, 32, 86, 111, 105, 45, 111, 25, 86, 36, 106, 108, 105]
